여러 페이지에 있는 데이터 가져오기

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

In [2]:
url = 'https://www.genie.co.kr/chart/top200'
header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.141 Whale/3.15.136.29 Safari/537.36'}
req = requests.get(url, headers = header)
soup = BeautifulSoup(req.text, 'html.parser')

In [ ]:
# 2~ 페이지
'''
https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220808&hh=10&rtm=Y&pg=1
https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220808&hh=10&rtm=Y&pg=2
https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220808&hh=10&rtm=Y&pg=3
https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220808&hh=10&rtm=Y&pg=4
'''

In [3]:
base_url = 'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220808&hh=10&rtm=Y&pg='
url = base_url + str(1)

원하는 목록 찾기

In [4]:
# tr 태그의 클래스명이 list일 때
trs = soup.select('tr.list')
#trs = soup.select('div > table > tbody > tr')
len(trs)

50

In [4]:
trs = soup.select('div > table > tbody > tr')
len(trs)

50

데이터 하나를 선택해 원하는 정보 추출

In [5]:
tr = trs[0]

In [6]:
rank = tr.select_one('.number').get_text()
rank

'1\n                                        \n                                    \n유지\n\n'

In [7]:
rank.split('\n')[0]

'1'

In [8]:
# 순위
rank = int(tr.select_one('.number').get_text().split('\n')[0])
rank

1

In [9]:
# 제목
title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
title

'그때 그 순간 그대로 (그그그)'

In [10]:
# 제목
artist = tr.select_one('.artist.ellipsis').get_text().strip()
artist

'WSG워너비 (가야G)'

In [11]:
# 앨범
album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
album

'WSG워너비 1집'

한 페이지 내의 정보

In [12]:
genie_list = []
for i in trs:
  rank = int(i.select_one('.number').get_text().split('\n')[0])
  title = i.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
  artist = i.select_one('.artist.ellipsis').get_text().strip()
  album = i.select_one('.albumtitle.ellipsis').get_text().strip()
  genie_list.append([rank, title, artist, album])

In [13]:
del genie_list

In [14]:
base_url

'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220808&hh=10&rtm=Y&pg='

In [16]:
genie_list = []

for page in range(1,5):
  url = base_url + str(page)
  req = requests.get(url, headers = header)
  soup = BeautifulSoup(req.text, 'html.parser')
  trs = soup.select('tr.list')
  for i in trs:
    rank = int(i.select_one('.number').get_text().split('\n')[0])
    title = i.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
    artist = i.select_one('.artist.ellipsis').get_text().strip()
    album = i.select_one('.albumtitle.ellipsis').get_text().strip()
    genie_list.append([rank, title, artist, album])

In [27]:
df = pd.DataFrame(genie_list, columns = ['순위', '제목', '가수', '앨범'])

In [29]:
df

,순위,제목,가수,앨범
0,1,그때 그 순간 그대로 (그그그),WSG워너비 (가야G),WSG워너비 1집
1,2,Attention,NewJeans,NewJeans 1st EP 'New Jeans'
2,3,보고싶었어,WSG워너비 (4FIRE),WSG워너비 1집
3,4,LOVE DIVE,IVE (아이브),LOVE DIVE
4,5,SNEAKERS,ITZY (있지),CHECKMATE
...,...,...,...,...
195,196,Shape Of You,Ed Sheeran,÷ (Deluxe)
196,197,Light Switch,Charlie Puth,That's Hilarious
197,198,Summer (Feat. BE'O (비오)),Paul Blanco,Summer
198,199,Savage Love (Laxed - Siren Beat) (BTS Remix),Jawsh 685 & Jason Derulo & 방탄소년단,Savage Love (Laxed - Siren Beat) (BTS Remix)


In [30]:
df.to_csv('Genie_Top200_220808.csv', index=False)